### 실패한 패스의 Intended-receiver레이블링 작업 통합

- 애니메이션 제작 후 레이블링 수행
- 두명의 연구원의 교차검증을 통해 레이블링 수행

In [6]:
import os, sys
import tqdm
sys.path.append(os.path.abspath(os.path.dirname(os.getcwd())))

In [7]:
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt

- PassExclude설명

1. (25개): 패스가 아님
2. (3개): 애매한 패스(패스인지 아닌지 불분명)
3. (2개): 데이터 자체의 오류

In [8]:
unsuccess_passes = pd.read_csv('../data/unified-data/unsuccess_passes.csv')
unsuccess_passes

,team,type,subtype,from,to,game_id,eventType,outcome,event_id,PassExclude,PassIntent
0,Away,BALL OUT,CLEARANCE,B16,NaN,1,PASS,False,77,1.0,NaN
1,Home,BALL OUT,CLEARANCE,A02,NaN,1,PASS,False,187,1.0,NaN
2,Away,BALL OUT,CLEARANCE,B16,NaN,1,PASS,False,272,1.0,NaN
3,Away,BALL LOST,CLEARANCE,B22,NaN,1,PASS,False,401,1.0,NaN
4,Away,BALL LOST,CLEARANCE,B19,NaN,1,PASS,False,417,NaN,B24
...,...,...,...,...,...,...,...,...,...,...,...
406,Home,BALL LOST,INTERCEPTION,B21,NaN,3,PASS,False,3318,NaN,B31
407,Away,BALL LOST,INTERCEPTION,A06,NaN,3,PASS,False,3319,NaN,A11
408,Home,BALL LOST,INTERCEPTION,B21,NaN,3,PASS,False,3331,NaN,B31
409,Away,BALL LOST,INTERCEPTION,A17,NaN,3,PASS,False,3335,NaN,A01


In [9]:
all_events = pd.read_csv("../data/unified-data/all-match.csv")
all_events

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,...,end_x,end_y,phase,game_id,is_left_to_right,goal,ownGoal,eventType,outcome,event_id
0,Away,PASS,PASS,1,1,0.04,3,0.12,B19,B21,...,59.40000,30.96000,1,1,False,False,False,PASS,True,0
1,Away,PASS,PASS,1,3,0.12,17,0.68,B21,B15,...,62.64000,15.12000,1,1,False,False,False,PASS,True,1
2,Away,PASS,PASS,1,45,1.80,61,2.44,B15,B19,...,48.60000,22.32000,1,1,False,False,False,PASS,True,2
3,Away,PASS,PASS,1,77,3.08,96,3.84,B19,B21,...,52.92000,33.84000,1,1,False,False,False,PASS,True,3
4,Away,PASS,PASS,1,191,7.64,217,8.68,B21,B22,...,34.56000,70.56000,1,1,False,False,False,PASS,True,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4774,Away,BALL LOST,CROSS,2,111643,4465.72,111669,4466.76,A01,NaN,...,103.68000,41.04000,9,3,True,False,False,CROSS,False,4774
4775,Away,BALL LOST,CROSS,2,112022,4480.88,112065,4482.60,A01,NaN,...,105.84000,34.56000,9,3,True,False,False,CROSS,False,4775
4776,Away,BALL LOST,CROSS,2,122324,4892.96,122362,4894.48,A13,NaN,...,104.76000,36.00000,10,3,True,False,False,CROSS,False,4776
4777,Away,PASS,CROSS,2,132718,5308.72,132764,5310.56,A13,A16,...,97.94844,29.87064,11,3,True,False,False,CROSS,True,4777


In [26]:
all_events[(all_events["eventType"].isin(["PASS","CROSS"])) & (~all_events["outcome"]) & (all_events["start_frame"] < all_events["end_frame"])]

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,...,end_x,end_y,phase,game_id,is_left_to_right,goal,ownGoal,eventType,outcome,event_id
6,Away,BALL LOST,INTERCEPTION,1,346,13.84,380,15.20,B17,NaN,...,29.16,54.00,1,1,False,False,False,PASS,False,6
7,Home,BALL LOST,INTERCEPTION,1,378,15.12,452,18.08,A02,NaN,...,63.72,46.08,1,1,True,False,False,PASS,False,7
10,Home,BALL LOST,INTERCEPTION,1,572,22.88,616,24.64,A10,NaN,...,72.36,31.68,1,1,True,False,False,PASS,False,10
15,Away,BALL LOST,INTERCEPTION,1,1110,44.40,1134,45.36,B17,NaN,...,33.48,60.48,1,1,False,False,False,PASS,False,15
19,Home,BALL LOST,INTERCEPTION,1,1370,54.80,1375,55.00,A08,NaN,...,95.04,20.16,1,1,True,False,False,PASS,False,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4772,Home,BALL OUT,CROSS,1,64175,2567.00,64202,2568.08,B26,NaN,...,110.16,36.00,1,3,True,False,False,CROSS,False,4772
4773,Home,BALL LOST,CROSS,2,86891,3475.64,86930,3477.20,B25,NaN,...,4.32,36.72,2,3,False,False,False,CROSS,False,4773
4774,Away,BALL LOST,CROSS,2,111643,4465.72,111669,4466.76,A01,NaN,...,103.68,41.04,9,3,True,False,False,CROSS,False,4774
4775,Away,BALL LOST,CROSS,2,112022,4480.88,112065,4482.60,A01,NaN,...,105.84,34.56,9,3,True,False,False,CROSS,False,4775


In [29]:
labeling_all_events = pd.merge(all_events, unsuccess_passes, on = "event_id", how="left", suffixes=("","_temp"))
labeling_all_events

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,...,team_temp,type_temp,subtype_temp,from_temp,to_temp,game_id_temp,eventType_temp,outcome_temp,PassExclude,PassIntent
0,Away,PASS,PASS,1,1,0.04,3,0.12,B19,B21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Away,PASS,PASS,1,3,0.12,17,0.68,B21,B15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Away,PASS,PASS,1,45,1.80,61,2.44,B15,B19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Away,PASS,PASS,1,77,3.08,96,3.84,B19,B21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Away,PASS,PASS,1,191,7.64,217,8.68,B21,B22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4774,Away,BALL LOST,CROSS,2,111643,4465.72,111669,4466.76,A01,NaN,...,Away,BALL LOST,CROSS,A01,NaN,3.0,CROSS,False,NaN,A13
4775,Away,BALL LOST,CROSS,2,112022,4480.88,112065,4482.60,A01,NaN,...,Away,BALL LOST,CROSS,A01,NaN,3.0,CROSS,False,NaN,A13
4776,Away,BALL LOST,CROSS,2,122324,4892.96,122362,4894.48,A13,NaN,...,Away,BALL LOST,CROSS,A13,NaN,3.0,CROSS,False,NaN,A14
4777,Away,PASS,CROSS,2,132718,5308.72,132764,5310.56,A13,A16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
col = [col for col in labeling_all_events.columns if not col.endswith("_temp")]
labeling_all_events[col]

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,...,phase,game_id,is_left_to_right,goal,ownGoal,eventType,outcome,event_id,PassExclude,PassIntent
0,Away,PASS,PASS,1,1,0.04,3,0.12,B19,B21,...,1,1,False,False,False,PASS,True,0,NaN,NaN
1,Away,PASS,PASS,1,3,0.12,17,0.68,B21,B15,...,1,1,False,False,False,PASS,True,1,NaN,NaN
2,Away,PASS,PASS,1,45,1.80,61,2.44,B15,B19,...,1,1,False,False,False,PASS,True,2,NaN,NaN
3,Away,PASS,PASS,1,77,3.08,96,3.84,B19,B21,...,1,1,False,False,False,PASS,True,3,NaN,NaN
4,Away,PASS,PASS,1,191,7.64,217,8.68,B21,B22,...,1,1,False,False,False,PASS,True,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4774,Away,BALL LOST,CROSS,2,111643,4465.72,111669,4466.76,A01,NaN,...,9,3,True,False,False,CROSS,False,4774,NaN,A13
4775,Away,BALL LOST,CROSS,2,112022,4480.88,112065,4482.60,A01,NaN,...,9,3,True,False,False,CROSS,False,4775,NaN,A13
4776,Away,BALL LOST,CROSS,2,122324,4892.96,122362,4894.48,A13,NaN,...,10,3,True,False,False,CROSS,False,4776,NaN,A14
4777,Away,PASS,CROSS,2,132718,5308.72,132764,5310.56,A13,A16,...,11,3,True,False,False,CROSS,True,4777,NaN,NaN


In [33]:
labeling_all_events[(labeling_all_events["eventType"].isin(["PASS","CROSS"])) & (~labeling_all_events["outcome"]) & (labeling_all_events["start_frame"] < labeling_all_events["end_frame"])]

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,...,team_temp,type_temp,subtype_temp,from_temp,to_temp,game_id_temp,eventType_temp,outcome_temp,PassExclude,PassIntent
6,Away,BALL LOST,INTERCEPTION,1,346,13.84,380,15.20,B17,NaN,...,Away,BALL LOST,INTERCEPTION,B17,NaN,1.0,PASS,False,NaN,B24
7,Home,BALL LOST,INTERCEPTION,1,378,15.12,452,18.08,A02,NaN,...,Home,BALL LOST,INTERCEPTION,A02,NaN,1.0,PASS,False,NaN,A10
10,Home,BALL LOST,INTERCEPTION,1,572,22.88,616,24.64,A10,NaN,...,Home,BALL LOST,INTERCEPTION,A10,NaN,1.0,PASS,False,NaN,A08
15,Away,BALL LOST,INTERCEPTION,1,1110,44.40,1134,45.36,B17,NaN,...,Away,BALL LOST,INTERCEPTION,B17,NaN,1.0,PASS,False,NaN,B21
19,Home,BALL LOST,INTERCEPTION,1,1370,54.80,1375,55.00,A08,NaN,...,Home,BALL LOST,INTERCEPTION,A08,NaN,1.0,PASS,False,NaN,A09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4772,Home,BALL OUT,CROSS,1,64175,2567.00,64202,2568.08,B26,NaN,...,Home,BALL OUT,CROSS,B26,NaN,3.0,CROSS,False,NaN,B27
4773,Home,BALL LOST,CROSS,2,86891,3475.64,86930,3477.20,B25,NaN,...,Home,BALL LOST,CROSS,B25,NaN,3.0,CROSS,False,NaN,B27
4774,Away,BALL LOST,CROSS,2,111643,4465.72,111669,4466.76,A01,NaN,...,Away,BALL LOST,CROSS,A01,NaN,3.0,CROSS,False,NaN,A13
4775,Away,BALL LOST,CROSS,2,112022,4480.88,112065,4482.60,A01,NaN,...,Away,BALL LOST,CROSS,A01,NaN,3.0,CROSS,False,NaN,A13


In [35]:
labeling_all_events[col].to_csv("../data/unified-data/labeling-all-match.csv", index = False)